## Test NLP  
by Jeremy Trullier  
#### Trying some NLP libraries and model
#### Writing functions to easily retrieve relevant data from our word doc

In [1]:
import os
import json

from enum import Enum
from pptx.util import Inches
from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE

In [2]:
from process_json import *
from slide_functions import *

In [3]:
datapath = '../data/'

In [4]:
files = os.listdir(datapath)
for filename in files:
    print(filename)

connard_de_virus.png
default_picture.jpg
Disney_json_ppt2.json
example.json
lego.json


In [24]:
filename = "Disney_json_ppt2.json"
jsonpath = datapath+filename

In [6]:
with open(jsonpath) as fjson:
    data = json.load(fjson)

In [7]:
json_slides = data['document_content']
for elem in json_slides:
    print(elem, "\n") 

{'type': 'level_0', 'content': [{'type': 'title_0', 'content': ['Walt Disney Company']}]} 

{'type': 'level_2', 'content': [{'type': 'title_2', 'content': ['La compagnie']}, {'type': 'plain_text', 'content': ["The Walt Disney Company est une entreprise amÃ©ricaine crÃ©Ã©e en 1923 par Walt Disney sous le nom Disney Brothers Studios. Elle est, en 2012, le premier groupe de divertissement au monde, prÃ©sent dans l'industrie des mÃ©dias (journaux, radios, tÃ©lÃ©vision, internet) avec des Ã©missions, des sÃ©ries tÃ©lÃ©visÃ©es, du cinÃ©ma avec des courts-mÃ©trages d'animation (dans lesquels figurent de cÃ©lÃ¨bres personnages tels que Mickey Mouse, Donald Duck, Dingo, etc.), des longs mÃ©trages d'animation et des films en prises de vues rÃ©elles, dans l'industrie du tourisme et de loisirs (parcs d'attractions dont Disneyland et Walt Disney World Resort, hÃ´tels, bateaux de croisiÃ¨re), ainsi que dans les produits dÃ©rivÃ©s de ses nombreuses productions (jeux et jouets, jeux vidÃ©o, vÃªtements

In [8]:
slide_elements = []
breakdown_json(json_slides, slide_elements)

In [21]:
def retrieve_content(list_slides):
    """
    Retrieves titles and plain texts from slide elements (can get them with breakdown_json)
    Parameters:
        - list_slides : list of data in json format
    """
    keywords = ['level_', 'title_', 'subtitle', 'header']
    doc_content = []
    sublist = []
    for content in list_slides:
        r = [content for keyword in keywords if keyword in content]
        if len(r)==0:
            if filling == True:
                sublist.append(content)
            if content == 'plain_text':
                doc_content.append(sublist)
                sublist = []
                filling = True
        elif len(r)>0:
            filling = False

    return doc_content[1:]

In [22]:
for item in retrieve_content(slide_elements):
    print(len(item), item,'\n')

1 ["The Walt Disney Company est une entreprise amÃ©ricaine crÃ©Ã©e en 1923 par Walt Disney sous le nom Disney Brothers Studios. Elle est, en 2012, le premier groupe de divertissement au monde, prÃ©sent dans l'industrie des mÃ©dias (journaux, radios, tÃ©lÃ©vision, internet) avec des Ã©missions, des sÃ©ries tÃ©lÃ©visÃ©es, du cinÃ©ma avec des courts-mÃ©trages d'animation (dans lesquels figurent de cÃ©lÃ¨bres personnages tels que Mickey Mouse, Donald Duck, Dingo, etc.), des longs mÃ©trages d'animation et des films en prises de vues rÃ©elles, dans l'industrie du tourisme et de loisirs (parcs d'attractions dont Disneyland et Walt Disney World Resort, hÃ´tels, bateaux de croisiÃ¨re), ainsi que dans les produits dÃ©rivÃ©s de ses nombreuses productions (jeux et jouets, jeux vidÃ©o, vÃªtements, etc.). Elle assure aussi la distribution de productions de tiers."] 

3 ['PrÃ©sident du directoire : Bob Chapek', 'Responsable Financier : Christine McCarthy', 'Adjoint de Robert Iger : Peter E. Murphy'] 

In [26]:
input_file = jsonpath
output_file = datapath+"nlp_"+filename
output_file

'../data/nlp_Disney_json_ppt2.json'

In [67]:
target_line = "Walt Disney Company"
new_line = '"42"'
with open(input_file, "r", encoding = "utf-8") as ifile:
    fcontent = ifile.read()

lines = fcontent.split("\n")

for i, line in enumerate(lines):
    if (target_line in line):
        if (line[len(line)-1] == ','):
            lines[i] = new_line+','
            continue
        lines[i] = new_line
        print(line)
        print("changed")
        print(lines[i])

fcontent = "\n".join(lines)

with open(output_file, "w", encoding = "utf-8") as ofile:
    ofile.write(fcontent)

						"Walt Disney Company"
changed
"42"
						"The Walt Disney Company est une entreprise américaine créée en 1923 par Walt Disney sous le nom Disney Brothers Studios. Elle est, en 2012, le premier groupe de divertissement au monde, présent dans l'industrie des médias (journaux, radios, télévision, internet) avec des émissions, des séries télévisées, du cinéma avec des courts-métrages d'animation (dans lesquels figurent de célèbres personnages tels que Mickey Mouse, Donald Duck, Dingo, etc.), des longs métrages d'animation et des films en prises de vues réelles, dans l'industrie du tourisme et de loisirs (parcs d'attractions dont Disneyland et Walt Disney World Resort, hôtels, bateaux de croisière), ainsi que dans les produits dérivés de ses nombreuses productions (jeux et jouets, jeux vidéo, vêtements, etc.). Elle assure aussi la distribution de productions de tiers."
changed
"42"
								"Le 23 novembre 2016, le site internet financier The Motley Fool annonce que la Walt Disney Comp

In [11]:
def retrieve_content_and_titles(list_slides, l):
    """
    Retrieves titles and plain texts from json file
    Parameters:
        - list_slides : list of data in json format
        - l : the list to be returned
    """
    if (type(list_slides) == type(list())):
        for value in list_slides:
            if type(value) == type(str()):
                l.append(value)
            else:
                retrieve_content_and_titles(value, l)
    elif (type(list_slides) == type(dict())):
        for key,value in list_slides.items():
            retrieve_content_and_titles(value, l)
    else:
        pass
        #print(type(list_slides))

testl= []
retrieve_content_and_titles(json_slides, testl)

In [12]:
for i in testl:
    print(i,"\n")

Walt Disney Company 

La compagnie 

The Walt Disney Company est une entreprise amÃ©ricaine crÃ©Ã©e en 1923 par Walt Disney sous le nom Disney Brothers Studios. Elle est, en 2012, le premier groupe de divertissement au monde, prÃ©sent dans l'industrie des mÃ©dias (journaux, radios, tÃ©lÃ©vision, internet) avec des Ã©missions, des sÃ©ries tÃ©lÃ©visÃ©es, du cinÃ©ma avec des courts-mÃ©trages d'animation (dans lesquels figurent de cÃ©lÃ¨bres personnages tels que Mickey Mouse, Donald Duck, Dingo, etc.), des longs mÃ©trages d'animation et des films en prises de vues rÃ©elles, dans l'industrie du tourisme et de loisirs (parcs d'attractions dont Disneyland et Walt Disney World Resort, hÃ´tels, bateaux de croisiÃ¨re), ainsi que dans les produits dÃ©rivÃ©s de ses nombreuses productions (jeux et jouets, jeux vidÃ©o, vÃªtements, etc.). Elle assure aussi la distribution de productions de tiers. 

Direction gÃ©nÃ©rale actuelle 

PrÃ©sident du directoire : Bob Chapek 

Responsable Financier : Christi

### Extraction based summurazation

In [13]:
from nlp_extraction import _run_article_summary

In [14]:
article = "Peter and Elizabeth took a taxi to attend the night party in the city. While in the party, Elizabeth collapsed and was rushed to the hospital. Since she was diagnosed with a brain injury, the doctor told Peter to stay besides her until she gets well. Therefore, Peter stayed with her at the hospital for 3 days without leaving."

In [15]:
summary_results = _run_article_summary(article)
print(summary_results)

 While in the party, Elizabeth collapsed and was rushed to the hospital. Therefore, Peter stayed with her at the hospital for 3 days without leaving.


### Bix

In [16]:
import gensim
import collections

import gensim.downloader as api

from gensim.models import Word2Vec
from gensim.summarization import summarize
from gensim.parsing.preprocessing import remove_stopwords

In [17]:
print("▶",summarize(article, word_count=20))

▶ Peter and Elizabeth took a taxi to attend the night party in the city.
Therefore, Peter stayed with her at the hospital for 3 days without leaving.
